## Vgg16BN Finetuning experienced with the batch_normalized fc layers

### Result: Can't get val_loss to come down below what we had for regular Vgg16
###              Currently unable to rewire the trained dense-layers back with the conv.
###              model to generate outputs.

In [1]:
import Executor

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [3]:
from Executor import *

In [6]:
executor = ExecutorBuilder().\
    with_runID("augmented.bn").\
    and_().\
    with_Vgg16BN().\
    and_().\
    train_batch_size(128). \
    and_(). \
    learn_rate(0.001).\
    and_().\
    data_on_path("../data/segmented/").\
    build()


Pretrained Vgg16BN model loaded.
Found 6166 images belonging to 3 classes.
Found 1461 images belonging to 3 classes.
initialized training data from: ../data/segmented/train
initialized validation data from: ../data/segmented/valid
found number of softmax classes: 3


In [8]:
#done..
executor.precompute_conv_model_outputs()
#executor.load_precomputed_conv_models()

precomputing conv. model outputs..
Found 6166 images belonging to 3 classes.
Found 1461 images belonging to 3 classes.
('models saved to files: ', 'precomputed_trn_features.augmented.bn.h5', ' and ', 'precomputed_val_features.augmented.bn.h5')
done.


<Executor.Executor instance at 0x7feb89107320>

In [9]:
executor.tune_softmax_layer_for_epochs(1)
# 6255/6255 [==============================] - 201s - loss: 2.3536 - acc: 0.4571 - val_loss: 1.1148 - val_acc: 0.5261


Epoch 1/1
6166/6166 [==============================] - 196s - loss: 2.5284 - acc: 0.4437 - val_loss: 1.3194 - val_acc: 0.5428
Vgg model finetuned.


<Executor.Executor instance at 0x7feb89107320>

In [10]:
fc_model = executor.get_rescaled_fc_model(new_dropout=0.5)

getting rescaled fc model...
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
done...
('updating dropout from: ', 0.5, ' to: ', 0.5)


In [11]:
for layer in fc_model.layers:
    layer.trainable = False

fc_model.layers[-1].trainable = True

In [12]:
opt = Adam(lr=0.001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6166 samples, validate on 1461 samples
Epoch 1/5
6166/6166 [==============================] - 3s - loss: 2.0961 - acc: 0.4804 - val_loss: 1.1515 - val_acc: 0.5305
Epoch 2/5
6166/6166 [==============================] - 3s - loss: 1.8620 - acc: 0.4921 - val_loss: 1.1346 - val_acc: 0.5270
Epoch 3/5
6166/6166 [==============================] - 3s - loss: 1.6369 - acc: 0.5079 - val_loss: 1.0316 - val_acc: 0.5702
Epoch 4/5
6166/6166 [==============================] - 3s - loss: 1.5240 - acc: 0.5206 - val_loss: 1.0047 - val_acc: 0.5530
Epoch 5/5
6166/6166 [==============================] - 3s - loss: 1.4184 - acc: 0.5187 - val_loss: 1.0139 - val_acc: 0.5592


In [13]:
# tune for 5 more times...
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6166 samples, validate on 1461 samples
Epoch 1/5
6166/6166 [==============================] - 3s - loss: 1.3403 - acc: 0.5281 - val_loss: 0.9845 - val_acc: 0.5489
Epoch 2/5
6166/6166 [==============================] - 3s - loss: 1.3580 - acc: 0.5321 - val_loss: 1.0587 - val_acc: 0.5181
Epoch 3/5
6166/6166 [==============================] - 3s - loss: 1.3051 - acc: 0.5368 - val_loss: 1.0207 - val_acc: 0.5387
Epoch 4/5
6166/6166 [==============================] - 3s - loss: 1.3111 - acc: 0.5339 - val_loss: 1.0407 - val_acc: 0.5332
Epoch 5/5
6166/6166 [==============================] - 3s - loss: 1.2697 - acc: 0.5352 - val_loss: 0.9545 - val_acc: 0.5763


In [14]:
# Tune for 5 more times with reduced learn_rate
opt = Adam(lr=0.001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6166 samples, validate on 1461 samples
Epoch 1/5
6166/6166 [==============================] - 3s - loss: 1.3191 - acc: 0.5307 - val_loss: 0.9819 - val_acc: 0.5743
Epoch 2/5
6166/6166 [==============================] - 3s - loss: 1.2422 - acc: 0.5431 - val_loss: 1.0011 - val_acc: 0.5729
Epoch 3/5
6166/6166 [==============================] - 3s - loss: 1.2821 - acc: 0.5339 - val_loss: 0.9648 - val_acc: 0.5770
Epoch 4/5
6166/6166 [==============================] - 3s - loss: 1.2665 - acc: 0.5365 - val_loss: 0.9705 - val_acc: 0.5743
Epoch 5/5
6166/6166 [==============================] - 3s - loss: 1.2779 - acc: 0.5370 - val_loss: 0.9896 - val_acc: 0.5510


In [15]:
opt = Adam(lr=0.0001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6166 samples, validate on 1461 samples
Epoch 1/5
6166/6166 [==============================] - 3s - loss: 1.1422 - acc: 0.5585 - val_loss: 0.9044 - val_acc: 0.5989
Epoch 2/5
6166/6166 [==============================] - 3s - loss: 1.0973 - acc: 0.5584 - val_loss: 0.8916 - val_acc: 0.5955
Epoch 3/5
6166/6166 [==============================] - 3s - loss: 1.0816 - acc: 0.5631 - val_loss: 0.8975 - val_acc: 0.5927
Epoch 4/5
6166/6166 [==============================] - 3s - loss: 1.0724 - acc: 0.5723 - val_loss: 0.8914 - val_acc: 0.5962
Epoch 5/5
6166/6166 [==============================] - 3s - loss: 1.0457 - acc: 0.5660 - val_loss: 0.8780 - val_acc: 0.6044


In [16]:
# 5 more epochs at same lr
print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))


fine tuning the rescaled fc model...
Train on 6166 samples, validate on 1461 samples
Epoch 1/5
6166/6166 [==============================] - 3s - loss: 1.0037 - acc: 0.5762 - val_loss: 0.8915 - val_acc: 0.5914
Epoch 2/5
6166/6166 [==============================] - 3s - loss: 1.0122 - acc: 0.5800 - val_loss: 0.8743 - val_acc: 0.6023
Epoch 3/5
6166/6166 [==============================] - 3s - loss: 0.9898 - acc: 0.5718 - val_loss: 0.8642 - val_acc: 0.5989
Epoch 4/5
6166/6166 [==============================] - 3s - loss: 0.9860 - acc: 0.5803 - val_loss: 0.8648 - val_acc: 0.6016
Epoch 5/5
6166/6166 [==============================] - 3s - loss: 0.9735 - acc: 0.5848 - val_loss: 0.8644 - val_acc: 0.5934


In [18]:
for layer in fc_model.layers:
    layer.trainable=True

fc_model.save_weights("fc_model_finetuned_val_loss_0_86.augmented.weights.bn.h5")

In [19]:
# final val_loss increases.. so decrease lr
opt = Adam(lr=0.00001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6166 samples, validate on 1461 samples
Epoch 1/5
6166/6166 [==============================] - 8s - loss: 0.9603 - acc: 0.5887 - val_loss: 0.8350 - val_acc: 0.6099
Epoch 2/5
6166/6166 [==============================] - 8s - loss: 0.7939 - acc: 0.6542 - val_loss: 0.7988 - val_acc: 0.6304
Epoch 3/5
6166/6166 [==============================] - 8s - loss: 0.6657 - acc: 0.7136 - val_loss: 0.7729 - val_acc: 0.6516
Epoch 4/5
6166/6166 [==============================] - 8s - loss: 0.5662 - acc: 0.7655 - val_loss: 0.7588 - val_acc: 0.6523
Epoch 5/5
6166/6166 [==============================] - 8s - loss: 0.4699 - acc: 0.8085 - val_loss: 0.7426 - val_acc: 0.6708


In [20]:
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6166 samples, validate on 1461 samples
Epoch 1/5
6166/6166 [==============================] - 8s - loss: 0.3770 - acc: 0.8500 - val_loss: 0.7308 - val_acc: 0.6715
Epoch 2/5
6166/6166 [==============================] - 8s - loss: 0.3113 - acc: 0.8831 - val_loss: 0.7228 - val_acc: 0.6790
Epoch 3/5
6166/6166 [==============================] - 9s - loss: 0.2716 - acc: 0.9048 - val_loss: 0.7250 - val_acc: 0.6886
Epoch 4/5
6166/6166 [==============================] - 9s - loss: 0.2194 - acc: 0.9262 - val_loss: 0.7224 - val_acc: 0.6886
Epoch 5/5
6166/6166 [==============================] - 9s - loss: 0.1776 - acc: 0.9428 - val_loss: 0.7254 - val_acc: 0.6879


In [21]:
opt = Adam(lr=0.000001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6166 samples, validate on 1461 samples
Epoch 1/5
6166/6166 [==============================] - 8s - loss: 0.1536 - acc: 0.9530 - val_loss: 0.7216 - val_acc: 0.6879
Epoch 2/5
6166/6166 [==============================] - 9s - loss: 0.1513 - acc: 0.9546 - val_loss: 0.7229 - val_acc: 0.6872
Epoch 3/5
6166/6166 [==============================] - 9s - loss: 0.1412 - acc: 0.9586 - val_loss: 0.7229 - val_acc: 0.6899
Epoch 4/5
6166/6166 [==============================] - 9s - loss: 0.1427 - acc: 0.9560 - val_loss: 0.7233 - val_acc: 0.6920
Epoch 5/5
6166/6166 [==============================] - 8s - loss: 0.1361 - acc: 0.9599 - val_loss: 0.7242 - val_acc: 0.6920


## Generate Predictions

In [22]:
fc_model.save_weights("fc_model_finetuned_val_loss_0_72.augmented.weights.bn.h5")

In [24]:
layers = executor.vgg.model.layers
last_conv_idx = [index for index, layer in enumerate(layers)
                 if type(layer) is Convolution2D][-1]
print("last conv index: ",last_conv_idx)

('last conv index: ', 30)


In [27]:

for l1,l2 in zip(executor.vgg.model.layers[last_conv_idx+1:],fc_model.layers):
    print("l1: ",l1, "l2: ",l2)
    l1.set_weights(l2.get_weights())

('l1: ', <keras.layers.pooling.MaxPooling2D object at 0x7feb8506f090>, 'l2: ', <keras.layers.pooling.MaxPooling2D object at 0x7feb64869fd0>)
('l1: ', <keras.layers.core.Flatten object at 0x7feb85002b90>, 'l2: ', <keras.layers.core.Flatten object at 0x7feb64869f10>)
('l1: ', <keras.layers.core.Dense object at 0x7feb8500bb50>, 'l2: ', <keras.layers.core.Dense object at 0x7feb84f994d0>)
('l1: ', <keras.layers.normalization.BatchNormalization object at 0x7feb85010e10>, 'l2: ', <keras.layers.normalization.BatchNormalization object at 0x7feb84f7bb50>)
('l1: ', <keras.layers.core.Dropout object at 0x7feb84f62190>, 'l2: ', <keras.layers.core.Dropout object at 0x7feb64866190>)
('l1: ', <keras.layers.core.Dense object at 0x7feb84f6e3d0>, 'l2: ', <keras.layers.core.Dense object at 0x7feb64866410>)
('l1: ', <keras.layers.normalization.BatchNormalization object at 0x7feb84f7b650>, 'l2: ', <keras.layers.normalization.BatchNormalization object at 0x7feb648660d0>)
('l1: ', <keras.layers.core.Dropout o

In [28]:
executor.build_predictions_on_test_data()

Found 512 images belonging to 1 classes.


<Executor.Executor instance at 0x7feb89107320>

In [29]:
executor.save_predictions_to_file("predictions.val_loss_0_72.bn.h5")

/home/ubuntu/kaggle_cervical_cancer_code/src/predictions.val_loss_0_72.bn.h5

In [31]:
fc_model.load_weights("fc_model_finetuned_val_loss_0_72.augmented.weights.bn.h5")

In [32]:
fc_model = executor.get_rescaled_fc_model(new_dropout=0.3)

getting rescaled fc model...
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
done...
('updating dropout from: ', 0.5, ' to: ', 0.3)


In [34]:
for layer in fc_model.layers:
    layer.trainable = False
fc_model.layers[-1].trainable = True

opt = Adam(lr=0.000001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=2,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6166 samples, validate on 1461 samples
Epoch 1/2
6166/6166 [==============================] - 3s - loss: 0.1872 - acc: 0.9924 - val_loss: 0.7558 - val_acc: 0.6920
Epoch 2/2
6166/6166 [==============================] - 3s - loss: 0.1855 - acc: 0.9929 - val_loss: 0.7217 - val_acc: 0.6913


In [35]:
opt = Adam(lr=0.00001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6166 samples, validate on 1461 samples
Epoch 1/5
6166/6166 [==============================] - 3s - loss: 0.1798 - acc: 0.9924 - val_loss: 0.6978 - val_acc: 0.6906
Epoch 2/5
6166/6166 [==============================] - 3s - loss: 0.1742 - acc: 0.9899 - val_loss: 0.6885 - val_acc: 0.6927
Epoch 3/5
6166/6166 [==============================] - 3s - loss: 0.1623 - acc: 0.9925 - val_loss: 0.6849 - val_acc: 0.6927
Epoch 4/5
6166/6166 [==============================] - 3s - loss: 0.1559 - acc: 0.9925 - val_loss: 0.6838 - val_acc: 0.6934
Epoch 5/5
6166/6166 [==============================] - 3s - loss: 0.1486 - acc: 0.9916 - val_loss: 0.6833 - val_acc: 0.6920


In [36]:
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6166 samples, validate on 1461 samples
Epoch 1/5
6166/6166 [==============================] - 3s - loss: 0.1460 - acc: 0.9912 - val_loss: 0.6850 - val_acc: 0.6886
Epoch 2/5
6166/6166 [==============================] - 3s - loss: 0.1436 - acc: 0.9904 - val_loss: 0.6865 - val_acc: 0.6927
Epoch 3/5
6166/6166 [==============================] - 3s - loss: 0.1385 - acc: 0.9899 - val_loss: 0.6885 - val_acc: 0.6940
Epoch 4/5
6166/6166 [==============================] - 3s - loss: 0.1352 - acc: 0.9908 - val_loss: 0.6891 - val_acc: 0.6947
Epoch 5/5
6166/6166 [==============================] - 3s - loss: 0.1303 - acc: 0.9929 - val_loss: 0.6917 - val_acc: 0.6927


In [37]:
fc_model.layers[-2].trainable = True

In [38]:
opt = Adam(lr=0.00001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))

fine tuning the rescaled fc model...
Train on 6166 samples, validate on 1461 samples
Epoch 1/5
6166/6166 [==============================] - 3s - loss: 0.1303 - acc: 0.9908 - val_loss: 0.6929 - val_acc: 0.6920
Epoch 2/5
6166/6166 [==============================] - 3s - loss: 0.1271 - acc: 0.9908 - val_loss: 0.6943 - val_acc: 0.6947
Epoch 3/5
6166/6166 [==============================] - 3s - loss: 0.1217 - acc: 0.9922 - val_loss: 0.6949 - val_acc: 0.6954
Epoch 4/5
6166/6166 [==============================] - 3s - loss: 0.1204 - acc: 0.9895 - val_loss: 0.6968 - val_acc: 0.6940
Epoch 5/5
6166/6166 [==============================] - 3s - loss: 0.1150 - acc: 0.9914 - val_loss: 0.6987 - val_acc: 0.6913


In [39]:
for layer in fc_model.layers:
    layer.trainable = True

opt = Adam(lr=0.000001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=2,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))    

fine tuning the rescaled fc model...
Train on 6166 samples, validate on 1461 samples
Epoch 1/2
6166/6166 [==============================] - 8s - loss: 0.1123 - acc: 0.9922 - val_loss: 0.6965 - val_acc: 0.6954
Epoch 2/2
6166/6166 [==============================] - 9s - loss: 0.0985 - acc: 0.9937 - val_loss: 0.6965 - val_acc: 0.6954


In [40]:
fc_model.save_weights("fc_model_finetuned_val_loss_0_69.augmented.weights.bn.h5")

In [41]:
for l1,l2 in zip(executor.vgg.model.layers[last_conv_idx+1:],fc_model.layers):
    l1.set_weights(l2.get_weights())

In [42]:
executor.build_predictions_on_test_data().and_().save_predictions_to_file("predictions.val_loss_0_69.bn.h5")

Found 512 images belonging to 1 classes.


/home/ubuntu/kaggle_cervical_cancer_code/src/predictions.val_loss_0_69.bn.h5

In [43]:
opt = Adam(lr=0.00001)
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


print("fine tuning the rescaled fc model...")
fc_model.fit(executor.train_precomputed, executor.train_labels, nb_epoch=5,
               batch_size= executor.batch_size, validation_data=(executor.val_precomputed, executor.val_labels))    

fine tuning the rescaled fc model...
Train on 6166 samples, validate on 1461 samples
Epoch 1/5
6166/6166 [==============================] - 8s - loss: 0.0906 - acc: 0.9906 - val_loss: 0.7166 - val_acc: 0.6920
Epoch 2/5
6166/6166 [==============================] - 8s - loss: 0.0353 - acc: 0.9985 - val_loss: 0.7125 - val_acc: 0.7091
Epoch 3/5
6166/6166 [==============================] - 9s - loss: 0.0183 - acc: 0.9994 - val_loss: 0.7195 - val_acc: 0.7146
Epoch 4/5
6166/6166 [==============================] - 8s - loss: 0.0121 - acc: 0.9995 - val_loss: 0.7491 - val_acc: 0.7098
Epoch 5/5
6166/6166 [==============================] - 9s - loss: 0.0088 - acc: 1.0000 - val_loss: 0.7634 - val_acc: 0.7118
